In [1]:
import pandas as pd
%matplotlib inline
import tangos
import tangos.examples.mergers as mergers

import numpy as np
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
h = 0.6776942783267969

from tangos_halo_module.halo_properties import get_main_progenitor_branch, track_halo_property, get_timesteps, ID_to_sim_halo_snap, infall_final_n_particles, infall_final_coordinates, apocentric_distance, disruption_time, accretion_time, orbit_interpolation, infall_velocity, quenching_time, max_sSFR_time, max_mass_time 
from tangos_halo_module.path import get_file_path, get_halo_snap_num, read_file
from tangos_halo_module.halos import ID_to_tangos_halo, get_survivors, get_zombies, get_host, get_survivor_IDs, get_zombie_IDs, blockPrint, enablePrint

In [9]:
data = pd.read_csv('Mint_Data/Data100_new.csv')
status = data['Status'].to_numpy()
ids = data['ID'].to_numpy()
sim = data['Simulation'].to_numpy()
t_quench = data['time_quench'].to_numpy()

In [14]:
len(ids[(sim=='h329')&(t_quench==-1)])

24

In [ ]:
data = pd.read_csv('Data.csv')
status = data['Status'].to_numpy()
ids = data['IDs'].to_numpy()
t_disrupt = data['time_disruption'].to_numpy()
max_Mstar = data['max_stellarmass'].to_numpy()
log_max_Mstar = np.log10(max_Mstar)

t_quench = np.array([quenching_time(ID=idx) for idx in ids]) 
t_infall = np.array([accretion_time(ID=idx)[0] for idx in ids]) 
 
lookback_t_quench = 13.80-t_quench
lookback_t_disrupt = 13.80-t_disrupt
lookback_t_infall = 13.80-t_infall

phi_infall = np.array([infall_final_coordinates(ID=idx)[0][4] for idx in ids])
phi_infall_deg = 180*phi_infall/np.pi
phi_infall_deg[phi_infall_deg>90]=180-phi_infall_deg[phi_infall_deg>90]

phi_final = np.array([infall_final_coordinates(ID=idx)[1][4] for idx in ids])
phi_final_deg = 180*phi_final/np.pi
phi_final_deg[phi_final_deg>90]=180-phi_final_deg[phi_final_deg>90]

Mstar_infall = np.array([accretion_time(ID=idx)[1] for idx in ids]) 
log_Mstar_infall = np.log10(Mstar_infall)

t_final = data['time_disruption'].to_numpy()
t_final[t_final==-1]=13.80 # For survivors
lookback_t_final = 13.80-t_final
Delta_t = t_final - t_infall

In [ ]:
# Color by stellar mass at infall
fig, ax1 = plt.subplots(1, 1, sharex='none', sharey='none', figsize=(10, 8))
fig.set_facecolor('w')
# fig.set_dpi(150.0)
np.seterr(divide='ignore')

linx=np.linspace(0, 14, 100)
cm = plt.cm.get_cmap('gnuplot')
# Increase weight of axes
plt.setp(ax1.spines.values(), linewidth=3)
x=lookback_t_infall
y=lookback_t_final
z=log_Mstar_infall
# Survivors
im1 = ax1.scatter(x[status=='Survivor'], y[status=='Survivor']+0.3, marker='*', edgecolor='k', c=z[status=='Survivor'], alpha=0.8, lw=1, s=300, cmap=cm, vmin=3, vmax=9, label='Surviving') 
ax1.scatter(x[status=='Zombie'], y[status=='Zombie'], marker='.', edgecolor='black', c=z[status=='Zombie'], alpha=0.8, lw=1, s=400, cmap=cm, vmin=3, vmax=9, label='Disrupted') 
ax1.plot(linx, linx, lw=0.7, c='black', ls='--')

cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Infall Stellar Mass [log $M_\odot$]', rotation=270, fontsize=23, labelpad=23)
cbar1.ax.tick_params(labelsize=20)

ax1.set_xlim(0, 14)
ax1.set_ylim(0, 14)
ax1.legend(loc='upper left', fontsize=18, frameon=True)
ax1.tick_params(which='major', length=15, width=2, labelsize=15)
ax1.tick_params(which='minor', length=8, width=2, labelsize=15)
ax1.tick_params(direction='in', which='both', labelsize=20, bottom=True, top=True, left=True, right=True)
ax1.set_title('Disruption Times', fontsize=20, pad=10)
ax1.set_xlabel('Lookback Time to Infall [Gyr]', fontsize=18)
ax1.set_ylabel('Lookback Time to Disruption [Gyr]', fontsize=18)

In [19]:
def get_main_progenitor_branch(tangos_halo, simulation, resolution=1000):
    '''
    input params: 
        tangos_halo: a valid Tangos halo object
        simulation: h148, h229, h242, h329: string
    output params:
        Main Progenitor Branch: 
                list of main progenitor branch halos from earliest to latest snapshot
        ids of Main Progenitor Branch:
                list of main progenitor branch halo ids from earliest to latest snapshot
    '''
    if resolution==1000:
        tangos.init_db("simulation_database/"+ str(simulation) +".db")
    elif resolution==100:
        tangos.init_db("/data/Sims/Databases/100Nptcls/"+ str(simulation) +".db")
    else:
        raise ValueError("Resolution not implemented yet.")
    ids = track_halo_property(simulation=simulation, key="halo_number()", 
                              tangos_halo=tangos_halo, halo_path=0, 
                              halo_id=0, snap_num=0, resolution=resolution) # earliest to latest timestep
    MPB = np.zeros(0)
    unique_ids=[] # earliest to most recent timestestep
    x = 0
    timesteps = tangos.get_simulation("snapshots").timesteps
    snapnums = get_timesteps(simulation=simulation, resolution=resolution)[3]
    while x <= len(timesteps)-1:
        x = int(x)
        # idx get_host= int(ids_flip[x]) 
        idx = int(ids[x])
        if idx == 0:
            MPB = np.concatenate((MPB, [-1]), axis=None)
            unique_ids.append(0)
        else:
            idk=str(simulation)[1:]+str(snapnums[x])+str(idx)
            MPB = np.concatenate((MPB, [timesteps[x][idx]]), axis=None)
            unique_ids.append(idk)
        x += 1
    return MPB, unique_ids


In [20]:
halo=ID_to_tangos_halo(ID=148208818, resolution=100)
get_main_progenitor_branch(tangos_halo=halo, simulation='h148', resolution=100)

(array([<Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000139/halo_25' | NDM=7936 Nstar=10 Ngas=2064>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000188/halo_29' | NDM=17080 Nstar=70 Ngas=4443>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000225/halo_30' | NDM=27025 Nstar=155 Ngas=6511>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000275/halo_34' | NDM=42312 Nstar=382 Ngas=9038>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000347/halo_28' | NDM=73673 Nstar=893 Ngas=15010>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000384/halo_21' | NDM=113654 Nstar=1361 Ngas=23831>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000456/halo_27' | NDM=119485 Nstar=2134 Ngas=22987>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000512/halo_30' | NDM=115523 Nstar=2426 Ngas=23500>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000640/halo_31' | NDM=119741 Nstar=3142 Ngas=26568>,
        <Halo 'snapshots/h148.cosmo50PLK.3072

In [4]:
def get_zombie_IDs(simulation, resolution=1000):
    '''
    input params: 
        simulation: h148, h229, h242, h329: string
    output params:
        zombie IDs: IDs of unique satellites (with at least 1 star) that disrupt before z=0: a list of custom halo IDs
    '''
#     from .halo_properties import get_main_progenitor_branch
#     from .path import get_halo_snap_num

    # For some very strange reason, the function keeps breaking at different spots
    # Unless these print statements exist where they currently do
    # So... This happened.
    blockPrint()
    print("This won't print")
    # And this will enable printing again
    # If you ever need it
    #enablePrint()
    
    # Now this is the actual code.
    # Note: Getting MPB corrupts halo object (god knows how)

    if resolution==1000:
        tangos.init_db("simulation_database/"+ str(simulation) +".db")
    elif resolution==100:
        tangos.init_db("/data/Sims/Databases/100Nptcls/"+ str(simulation) +".db")
    else:
        raise ValueError("Resolution not implemented yet.")
    t_Gyr = get_timesteps(simulation=simulation, resolution=resolution)[0]
    print(t_Gyr)
    # Host
    host = get_host(simulation=simulation, resolution=resolution)
    print(host)

    # Zombies
    redshift, ratio, progenitor_halos = mergers.get_mergers_of_major_progenitor(host)
    zombies = [x[1] for x in progenitor_halos] 
    print(zombies)
    host_MPB, host_MPB_ids = get_main_progenitor_branch(tangos_halo=host, simulation=simulation, resolution=resolution)
    print(host_MPB_ids)
    latest_signature = []
    for sat in zombies: 
        print(sat['Mvir'])
        print(sat)
        sat_MPB, sat_MPB_ids = get_main_progenitor_branch(tangos_halo=sat, simulation=simulation, 
                                                          resolution=resolution)
        print(sat_MPB_ids)
        print(sat_MPB)
        mismatch_times = t_Gyr[(sat_MPB_ids!=host_MPB_ids)&(sat_MPB_ids!=0)]
        if len(mismatch_times)>0:
            latest_signature_time = max(mismatch_times)
            print(latest_signature_time)
            latest_sig = sat_MPB[(t_Gyr==latest_signature_time)]
            
            if len(str(latest_sig))>10: # check if it is not None
                print(latest_sig)
                latest_signature.append(latest_sig[0])

    IDs = []        
    for sat in latest_signature:
        IDs.append(get_halo_snap_num(tangos_halo=sat)[2])
    IDs = list(set(IDs))
    host_ID = get_halo_snap_num(tangos_halo=host)[2]
    if host_ID in IDs:
        IDs.remove(host_ID)
    #IDs
    IDs.sort()
    enablePrint()
    return IDs


In [4]:
get_zombie_IDs(simulation='h329', resolution=100)

['329010725',
 '3290192183',
 '329019237',
 '329019269',
 '32901927',
 '329019277',
 '329019291',
 '3290225115',
 '3290225183',
 '329022534',
 '329022538',
 '32902254',
 '329022566',
 '329022590',
 '329022595',
 '329027510',
 '329027534',
 '329028850',
 '329028853',
 '329028867',
 '329028868',
 '329034772',
 '329038418',
 '329038429',
 '329038440',
 '329038446',
 '329048010',
 '329048022',
 '329048025',
 '329048036',
 '329048070',
 '32905762',
 '32905769',
 '329063711',
 '329067212',
 '329067213',
 '329067220',
 '32906727',
 '329077610',
 '32907767',
 '329086413',
 '32908649',
 '32909748',
 '32911527',
 '329124811',
 '32912695',
 '32913443',
 '32914757',
 '32915364',
 '32923043',
 '32924962',
 '32928803',
 '32930722',
 '32931684',
 '32934566']

In [5]:
get_survivors(simulation='h229', resolution=100)

[<Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_10' | NDM=83829 Nstar=4642 Ngas=3572>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_1327' | NDM=549 Nstar=48 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_1381' | NDM=570 Nstar=3 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_18' | NDM=35419 Nstar=15681 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_191' | NDM=4037 Nstar=120 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_24' | NDM=37041 Nstar=973 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_3661' | NDM=178 Nstar=10 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_45' | NDM=19396 Nstar=1379 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_46' | NDM=20403 Nstar=127 Ngas=0>,
 <Halo 'snapshots/h229.cosmo50PLK.3072gst5HbwK1BH.004096/halo_48' | NDM=20082 Nstar=93 Ngas=0>,
 <Halo 'snapshots/h229.cosmo

In [26]:
# 
testhalo = ID_to_tangos_halo(ID='32934566', resolution=100)
print(testhalo)
get_halo_snap_num(tangos_halo=testhalo)

('6', '3456', '32934566', 'h329')

In [2]:
testhalo = ID_to_tangos_halo(ID=14840961101, resolution=100)
get_main_progenitor_branch(tangos_halo=testhalo, simulation='h148', resolution=100)

(array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
        -1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001269/halo_475' | NDM=4782 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001280/halo_489' | NDM=4634 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001408/halo_630' | NDM=3245 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001536/halo_731' | NDM=2694 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001740/halo_709' | NDM=2662 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001920/halo_680' | NDM=2720 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002048/halo_636' | NDM=2801 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002088/halo_633' | NDM=2804 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002176/halo_654' | 

In [8]:
get_survivor_IDs(simulation='h148', resolution=100)

['148409610',
 '148409610270',
 '14840961090',
 '14840961101',
 '148409612',
 '14840961292',
 '148409613',
 '1484096189',
 '14840962',
 '1484096217',
 '1484096223',
 '14840962297',
 '148409623',
 '1484096263',
 '1484096290',
 '14840963',
 '1484096306',
 '148409636',
 '1484096380',
 '148409639',
 '14840964',
 '1484096438',
 '148409644',
 '148409648',
 '1484096563',
 '148409662',
 '148409669',
 '1484096779',
 '1484096932']

In [4]:
tangos.init_db("/data/Sims/Databases/100Nptcls/h329.db")
timesteps = tangos.get_simulation("snapshots").timesteps 

In [7]:
timesteps[-1][1]

<Halo 'snapshots/h329.cosmo50PLK.3072gst5HbwK1BH.004096/halo_1' | NDM=14013447 Nstar=17592280 Ngas=1062105>

In [6]:
def track_halo_property(simulation, key, tangos_halo=0, halo_path=0, halo_id=0, snap_num=0, resolution=1000):
    '''
    input params: 
        simulation: h148, h229, h242, h329: string
        key: pre-existing tangos halo property (eg. 'Mvir' or 'VXc'): string
        tangos_halo: a valid Tangos halo object
        halo_path: 0 or a complete tangos halo address: path or string object
        halo_id: halo id: string or numeric
        snap_num: 4 digit simulation snapshot number: string
    output params:
        tracked parameter: earliest --> latest snapshot: numpy array
    '''
    
    if tangos_halo:
        halo = tangos_halo
    elif halo_id and snap_num and simulation:
        #string tangos halo query from components
        if simulation == 'h148':
            halo = tangos.get_halo("snapshots/"+ str(simulation) +
                               ".cosmo50PLK.3072g3HbwK1BH.00"+ str(snap_num) +"/"+ str(halo_id))
        else:
            halo = tangos.get_halo("snapshots/"+ str(simulation) +
                               ".cosmo50PLK.3072gst5HbwK1BH.00"+ str(snap_num) +"/"+ str(halo_id))
    elif halo_path:
        halo = tangos.get_halo(str(halo))
    else:
        raise ValueError("Halo %r not found. You got this tho." % (halo))

        #edit .db path to match local address
    if resolution==1000:
        tangos.init_db("simulation_database/"+ str(simulation) +".db")
    elif resolution==100:
        tangos.init_db("/data/Sims/Databases/100Nptcls/"+ str(simulation) +".db")
    else:
        raise ValueError("Resolution not implemented yet.")
        
    # round all times in Gyr to 2 decimal places due to 
    # inconsistent float truncation between response to livecalculation query and direct database query
    halo_gyr = round(halo.timestep.time_gyr, 2)
    prog = np.flip(halo.calculate_for_progenitors(str(key))[0])
    desc = halo.calculate_for_descendants(str(key))[0][1:]
    track = np.array(np.concatenate((prog, desc), axis=None))

    len_track = len(track)
    timesteps = get_timesteps(simulation)[0] 
    timesteps = [round(t, 2) for t in timesteps] #in gyr since start of sim
    num_timesteps = len(timesteps)
    #len(tangos.get_simulation("snapshots").timesteps) #62 for h229, h242, h329
    
    if len_track == num_timesteps:
        return np.asarray(track)
    elif len_track > num_timesteps/2: #pad track with initial 0s
        return np.array(np.concatenate((np.zeros(num_timesteps - len_track), track), axis=None), dtype=float)
    else:
        # i = np.where(timesteps==halo_gyr)[0][0]
        i = timesteps.index(halo_gyr)
        new_track = np.zeros(num_timesteps)
        new_track[i+1:i+1+len(desc)] = desc
        new_track[i+1-len(prog):i+1] = prog
        return new_track 
    

In [7]:
halo = ID_to_tangos_halo(ID='14840961', resolution=100)
track_halo_property(simulation='h148', key='Xc', tangos_halo=halo, halo_path=0, halo_id=0, snap_num=0, resolution=100)


array([17250.04846327, 17224.12010416, 17205.74227812, 17183.52993609,
       17153.04448364, 17134.29186282, 17092.54103052, 17079.01651849,
       17011.57054883, 16990.35823642, 16977.07445752, 16997.83137774,
       17003.22530367, 17019.32485121, 17029.55816248, 17051.83650939,
       17064.86364515, 17094.91266587, 17097.51465106, 17125.62652772,
       17122.75915771, 17128.21424544, 17126.48023106, 17123.67568034,
       17123.36464989, 17123.11332402, 17122.26981446, 17121.52016358,
       17125.521444  , 17134.82374467, 17138.39323282, 17139.98356925,
       17137.73082025, 17140.63340119, 17140.78567167, 17140.78471346,
       17137.96355394, 17134.53996007, 17133.65353969, 17131.02729267,
       17127.47658492, 17125.0676083 , 17123.35160689])

In [ ]:
for idx in get_survivor_IDs(simulation='h148', resolution=100):
    halo = ID_to_tangos_halo(ID=idx, resolution=100)
    print(halo)
    print(track_halo_property(simulation='h148', key='Zc', tangos_halo=halo, halo_path=0, halo_id=0, snap_num=0, resolution=100))

In [6]:
def get_main_progenitor_branch(tangos_halo, simulation, resolution=1000):
    '''
    input params: 
        tangos_halo: a valid Tangos halo object
        simulation: h148, h229, h242, h329: string
    output params:
        Main Progenitor Branch: 
                list of main progenitor branch halos from earliest to latest snapshot
        ids of Main Progenitor Branch:
                list of main progenitor branch halo ids from earliest to latest snapshot
    '''
    blockPrint()
    if resolution==1000:
        tangos.init_db("simulation_database/"+ str(simulation) +".db")
    elif resolution==100:
        tangos.init_db("/data/Sims/Databases/100Nptcls/"+ str(simulation) +".db")
    else:
        raise ValueError("Resolution not implemented yet.")
    ids = track_halo_property(simulation=simulation, key="halo_number()", 
                              tangos_halo=tangos_halo, halo_path=0, 
                              halo_id=0, snap_num=0, resolution=resolution) # earliest to latest timestep
    MPB = np.zeros(0) # earliest to most recent timestestep
    x = 0
    timesteps = get_timesteps(simulation=simulation, resolution=resolution)[3]
    while x <= len(timesteps)-1:
        x = int(x)
        idx = int(ids[x])
        if idx == 0:
            MPB = np.concatenate((MPB, [-1]), axis=None)
        else:
            idk=str(simulation)[1:]+str(int(timesteps[x]))+str(idx)
            halo=ID_to_tangos_halo(ID=idk, resolution=resolution)
            print(halo)
            MPB = np.concatenate((MPB, [halo]), axis=None)
        x += 1
    enablePrint()
    return MPB, ids

In [7]:
testhalo = ID_to_tangos_halo(ID='14840961101', resolution=100)
get_main_progenitor_branch(tangos_halo=testhalo, simulation='h148', resolution=100)

(array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
        -1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001269/halo_475' | NDM=4782 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001280/halo_489' | NDM=4634 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001408/halo_630' | NDM=3245 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001536/halo_731' | NDM=2694 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001740/halo_709' | NDM=2662 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.001920/halo_680' | NDM=2720 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002048/halo_636' | NDM=2801 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002088/halo_633' | NDM=2804 Nstar=21 Ngas=0>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.002176/halo_654' | 

In [22]:
simulation='h148'
tangos.init_db("/data/Sims/Databases/100Nptcls/"+ str(simulation) +".db")
timesteps = tangos.get_simulation("snapshots").timesteps

In [18]:
get_timesteps(simulation='h329', resolution=100)[3]

['0071',
 '0096',
 '0107',
 '0139',
 '0188',
 '0192',
 '0225',
 '0275',
 '0288',
 '0347',
 '0384',
 '0456',
 '0480',
 '0576',
 '0637',
 '0672',
 '0768',
 '0776',
 '0864',
 '0960',
 '0974',
 '1056',
 '1106',
 '1152',
 '1248',
 '1269',
 '1344',
 '1440',
 '1475',
 '1536',
 '1632',
 '1728',
 '1740',
 '1824',
 '1920',
 '2016',
 '2088',
 '2112',
 '2208',
 '2304',
 '2400',
 '2496',
 '2554',
 '2592',
 '2688',
 '2784',
 '2880',
 '2976',
 '3072',
 '3168',
 '3195',
 '3264',
 '3360',
 '3456',
 '3552',
 '3606',
 '3648',
 '3744',
 '3840',
 '3936',
 '4032',
 '4096']